In [1]:

from tensorflow import keras as keras

from drowsiness_detection import config
from drowsiness_detection.data import load_experiment_data
from drowsiness_detection.models import ThreeDStandardScaler
import matplotlib.pyplot as plt
import numpy as np
from contextlib import redirect_stdout
from sklearn.metrics import classification_report
from drowsiness_detection.data import session_type_mapping, label_names_dict

from dl_ts_models.main import create_classifier

In [2]:
# load data
config.set_paths(30, 10)
X_train, X_test, y_train, y_test, cv_splits = load_experiment_data(
    exclude_by="a",
    num_targets=2,
    seed=45,
    test_size=.2,
    split_by_subjects=True,
    use_dummy_data=False,
    nn_experiment=True,
    feature_col_indices=(5, 8, 9, 14, 15, 16, 19),
    model_name="mvts_transformer")

X_train shape: (29012, 300, 7), y_train shape: (29012,)
X_test shape: (7094, 300, 7), y_test shape: (7094,)


In [3]:
num_samples = 2000
X_train = X_train[:num_samples]
X_test = X_test[:num_samples]
y_test = y_test[:num_samples]
y_train = y_train[:num_samples]



In [4]:
scaler = ThreeDStandardScaler(feature_axis=-1)

X_train_scaled = scaler.fit_transform(X_train, y_train)
X_test_scaled = scaler.transform(X_test)

input_shape = (None, 300, 7)


In [5]:
model = create_classifier(
    classifier_name="resnet",
    input_shape=input_shape[1:],
    nb_classes=1,
    output_directory=str(config.SOURCES_ROOT_PATH.parent.joinpath("data/resnet_experiments")) + "/",
    verbose=True,
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 7)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 300, 64)      3648        ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 300, 64)     256         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 300, 64)      0           ['batch_normalization[0][0]']

In [6]:

metrics = model.fit(x_train=X_train_scaled, y_train=y_train, x_val=X_test_scaled, y_val=y_test, nb_epochs=2, batch_size=64,
                        class_weight={0: 0.84, 1: 1.14})
print(metrics)

Epoch 1/2
32/32 [==============================] - 25s 701ms/step - loss: 0.3805 - accuracy: 0.5590 - val_loss: 0.9470 - val_accuracy: 0.4135 - lr: 0.0010
Epoch 2/2
32/32 [==============================] - 22s 697ms/step - loss: 0.2137 - accuracy: 0.5590 - val_loss: 1.0224 - val_accuracy: 0.4135 - lr: 0.0010


/home/tim/IM/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial


[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
saving results to  /home/tim/IM/data/resnet_experiments/
   precision  accuracy  recall   duration
0    0.20675    0.4135     0.5  46.794905


In [ ]:
y_pred = model.predict(X_test_scaled, y_test)

In [ ]:
np.argmax([[0], [1]], axis=1)

In [ ]:
from sklearn.metrics import RocCurveDisplay

y_hat = model.predict(X_test_scaled)

RocCurveDisplay.from_predictions( y_pred=y_hat, y_true=y_test, pos_label=1)